In [41]:
import numpy as np
import os
import pandas as pd
import re

In [57]:
def list_dir(path):
    files = os.listdir(path)
    return [string for string in files if not string.startswith('.')]


def year2szn(year):
    return f"{str(year-1)}-{str(year)[-2:]}"


def combine_csvs(data_folder):
    ''' Group all csvs by team
        CSVs included are:
        - szn_totals.csv
        - advanced.csv
        - shooting.csv
        - per100poss.csv
        - standing.csv
        - szn_totals_against.csv'''
    dfs = []
    file_names = ['szn_totals.csv','szn_totals_against.csv','advanced.csv',
                  'shooting.csv','shooting_against.csv',
                  'szn_per100poss.csv','szn_per100poss_against.csv', 'standings.csv'
                 ]
    merge_column = 'Team'
    for subfolder in list_dir(data_folder):
        folderpath = os.path.join(data_folder, subfolder)
        if os.path.isdir(folderpath):
            csv_path = os.path.join(folderpath, file_names[0])
            # Check if 'szn_totals.csv' exists in the current subfolder
            if os.path.exists(csv_path):
                # Read the CSV file into a pandas DataFrame
                merged_df = pd.read_csv(csv_path)
                merged_df[merge_column] = [x.strip('*') for x in merged_df[merge_column]]
                #loop through rest of files
                for file_name in file_names[1:]:
                    # Read the current file
                    current_csv_path = os.path.join(folderpath, file_name)
                    if os.path.exists(current_csv_path):
                        current_df = pd.read_csv(current_csv_path)
                        if 'Team' in current_df.iloc[0].values:
                            current_df.columns = current_df.iloc[0]
                            current_df.drop(current_df.index[0], inplace=True)  
                        # Merge the current DataFrame with the merged DataFrame
                        if subfolder=='2022-23':
                            current_df[merge_column] = [x[:-5] for x in current_df[merge_column]]
                        merged_df = pd.merge(merged_df, current_df, on=merge_column, how='outer',suffixes=('', '_'+file_name[:-4]))
    
                # Add a new column with the year
                merged_df['Year'] = subfolder
                
                merged_df = merged_df.drop(merged_df.columns[0], axis=1)
                merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
                merged_df = merged_df.loc[:, merged_df.columns.notna()]
                merged_df = merged_df.drop(merged_df.columns[merged_df.columns.str.contains('^Unnamed:')], axis=1)
                
                merged_df = merged_df.reset_index(drop=True)
                # Append the modified DataFrame to the list
                dfs.append(merged_df)

    # Concatenate all the DataFrames in the list
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df = concatenated_df.sort_values(['Year', 'Rk']).reset_index(drop=True)
    return concatenated_df


def gather_years(year: int, previous_years: int) -> tuple[pd.DataFrame, pd.Series]:
    ''' Create a function that uses the current year as its validation set and previous years
    as training data'''
    year_string = year2szn(year)  # 2023 -> "2022-23"
    csvs_val = listdir(f'./data/{year_string}')
    y = gather_years(csvs_val)
    for i in range(1, previous_years):
        cur_year = year-i
        
    X = ...
    return X, y

In [58]:
all_years = combine_csvs('./data/')

In [59]:
all_years.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,2P_shooting_against,0-3_shooting_against,3-10_shooting_against,10-16_shooting_against,16-3P_shooting_against,3P_shooting_against,%FGA_shooting_against,Md._shooting_against,%3PA_shooting_against,3P%_shooting_against
0,1.0,Boston Celtics,69.0,NaN,2065.0,5607.0,0.368,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Rochester Royals,68.0,NaN,2032.0,5377.0,0.378,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Tri-Cities Blackhawks,68.0,NaN,1988.0,6041.0,0.329,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Fort Wayne Pistons,68.0,NaN,2002.0,5927.0,0.338,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Syracuse Nationals,66.0,NaN,1884.0,5365.0,0.351,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
for thing in list_dir('./data/2020-21/'):
    frame = pd.read_csv(f'./data/2020-21/{thing}')
    print((frame.columns), thing)

Index(['Unnamed: 0', 'Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object') szn_totals_against.csv
Index(['Unnamed: 0', 'Unnamed: 0_level_0', 'Unnamed: 1_level_0',
       'Unnamed: 2_level_0', 'Unnamed: 3_level_0', 'Unnamed: 4_level_0',
       'Unnamed: 5_level_0', 'Unnamed: 6_level_0', 'Unnamed: 7_level_0',
       'Unnamed: 8_level_0', 'Unnamed: 9_level_0', 'Unnamed: 10_level_0',
       'Unnamed: 11_level_0', 'Unnamed: 12_level_0', 'Unnamed: 13_level_0',
       'Unnamed: 14_level_0', 'Unnamed: 15_level_0', 'Unnamed: 16_level_0',
       'Unnamed: 17_level_0', 'Offense Four Factors', 'Offense Four Factors.1',
       'Offense Four Factors.2', 'Offense Four Factors.3',
       'Unnamed: 22_level_0', 'Defense Four Factors', 'Defense Four Factors.1',
       'Defense Four Factors.2', 'Defense Four Factors.3',
       'Unnamed: 27_level_0', 'Unname

In [53]:
def show_list(df) -> None:
    '''prints each columns name and its respective index in df.columns'''
    for x in range(len(df.columns)):
        print(x, '-', df.columns[x])

In [62]:
all_years.to_csv('./training.csv')

In [61]:
!pwd

/Users/alexbradshaw/Desktop/0DL/predict-perform
